In [1]:
import pandas as pd 
from sklearn.model_selection import train_test_split 
import os

In [2]:
data_df = pd.read_csv(r'C:\Users\viral\Desktop\final_project\myproject\dataset\label.csv')
print(data_df.dtypes)

def load_data():
    """
    Load training data and split it into training and validation set
    
    """
    #reads CSV file into a single dataframe variable
    data_df = pd.read_csv(r'C:\Users\viral\Desktop\final_project\myproject\dataset\label.csv', names=['img', 'ang', 'date', 'time'])
    #yay dataframes, we can select rows and columns by their names
    #we'll store the camera images as our input data
    X = data_df['img'].values
    #and our steering commands as our output data
    y = data_df['ang'].values
    
    #now we can split the data into a training (80), testing(20), and validation set
    #thanks scikit learn
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2,random_state=0)
    return X_train, X_valid, y_train, y_valid


img      object
ang     float64
date     object
time     object
dtype: object


In [3]:
from tensorflow.keras import layers
from tensorflow.keras import Model

In [9]:
import numpy as np
def build_model():
    """
    NVIDIA model used
    Image normalization to avoid saturation and make gradients work better.
    Convolution: 5x5, filter: 24, strides: 2x2, activation: ELU
    Convolution: 5x5, filter: 36, strides: 2x2, activation: ELU
    Convolution: 5x5, filter: 48, strides: 2x2, activation: ELU
    Convolution: 3x3, filter: 64, strides: 1x1, activation: ELU
    Convolution: 3x3, filter: 64, strides: 1x1, activation: ELU
    Drop out (0.5)
    Fully connected: neurons: 100, activation: ELU
    Fully connected: neurons: 50, activation: ELU
    Fully connected: neurons: 10, activation: ELU
    Fully connected: neurons: 1 (output)

    # the convolution layers are meant to handle feature engineering
    the fully connected layer for predicting the steering angle.
    dropout avoids overfitting
    ELU(Exponential linear unit) function takes care of the Vanishing gradient problem. 
    """
    # Our input feature map is 150x150x3: 150x150 for the image pixels, and 3 for
    # the three color channels: R, G, and B
   

    img_input = layers.Input(shape=(66, 200, 3))
    x = layers.Conv2D(24, 5, (2, 2), activation='relu')(img_input)
    #x = layers.MaxPooling2D(2)(x)
    x = layers.Conv2D(36, 5,(2, 2), activation='relu')(x)
    #x = layers.MaxPooling2D(2)(x)
    x = layers.Conv2D(48, 5,(2, 2), activation='relu')(x)
    #x = layers.MaxPooling2D(2)(x)
    x = layers.Conv2D(64, 3,(1, 1), activation='relu')(x)
    #x = layers.MaxPooling2D(1)(x)
    x = layers.Conv2D(64,1, (2, 2), activation='relu')(x)
    #x = layers.MaxPooling2D(2)(x)
    x = layers.Flatten()(x)
    x = layers.Dense(100, activation='relu')(x)
    x = layers.Dense(50, activation='relu')(x)
    x = layers.Dense(10, activation='relu')(x)
    output = layers.Dense(1)(x)
    model = Model(img_input, output)
    model.summary()
    
    return model



In [10]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint

In [11]:
def train_model(model, X_train, X_valid, y_train, y_valid):
    """
    Train the model
    """
    train_datagen = ImageDataGenerator(rescale=1./255)
    val_datagen = ImageDataGenerator(rescale=1./255)

    # Flow training images in batches of 20 using train_datagen generator
    train_generator = train_datagen.flow_from_dataframe(
            dataframe=data_df[:len(X_train)],
            directory=None,
            x_col='img',
            y_col='ang',
            target_size=(66, 200),
            classes=None,
            class_mode="raw",
            batch_size=100,
            save_format="jpg",
        )

    # Flow validation images in batches of 20 using val_datagen generator
    valid_generator = val_datagen.flow_from_dataframe(
            dataframe=data_df[len(X_train):],
            directory=None,
            x_col='img',
            y_col='ang',
            target_size=(66, 200),
            classes=None,
            class_mode="raw",
            batch_size=100,
            save_format="jpg",
        )
    print(train_generator)
   
    filepath_f = './Autopilot.h5'
    
    
    
    model.compile(loss='mean_squared_error', optimizer='Adam',metrics=['accuracy'])
    
    
    model.fit_generator(train_generator,
                        steps_per_epoch=300,
                        epochs=10,
                        max_queue_size=1,
                        validation_data=valid_generator,
                        validation_steps=50,
                        verbose=1)
    
    


#for command line args

In [12]:
def main():
   

    #load data
    data = load_data()
    #build model
    model = build_model()
    #train model on data, it saves as model.h5 
    train_model(model, *data)
    model.save('Autopilot.h5')


In [13]:
if __name__ == '__main__':
    main()

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 66, 200, 3)]      0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 31, 98, 24)        1824      
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 14, 47, 36)        21636     
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 5, 22, 48)         43248     
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 3, 20, 64)         27712     
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 2, 10, 64)         4160      
_________________________________________________________________
flatten_1 (Flatten)          (None, 1280)             